# Generate Input Data

1. Add operation and database identifier

In [1]:
import csv
import random

folder = "src/main/resources/data"

# file needs update operations (in dependent order)
files = ['nation.tbl', 'customer.tbl', 'orders.tbl', 'lineitem.tbl']

# number of updates
n_updates = 0.1

# read the file into a list
def read_file(file):
    with open(folder + "/" + file, 'r') as tbl_file:
        tbl_reader = csv.reader(tbl_file, delimiter='|')
        return list(tbl_reader)

# select a subset of the data for updates
def select_updates(data, n_updates):
    if n_updates < 1:
        n_updates = int(n_updates * len(data))
    else:
        n_updates = int(n_updates)
    updates = random.sample(data, n_updates)
    return updates

init_ops = []
update_ops = []

for file in files:
    print("Processing " + file)
    data = read_file(file)

    # select a subset of the data for updates
    updates = select_updates(data, n_updates)
    n_invalid_ops = max(1, int(len(updates)*0.5))
    
    # insert: +, delete: -
    insert_tag = f"+|{file.split('.')[0]}|"
    delete_tag = f"-|{file.split('.')[0]}|"
    # initialize the database (insert all tuples)
    init_ops += [insert_tag+'|'.join(i) for i in data]

    if file != "nation.tbl":
        # update operations
        update_ops += [insert_tag+'|'.join(u) for u in updates[:n_invalid_ops]] # invalid insertions
        update_ops += [delete_tag+'|'.join(u) for u in updates] # valid deletions
        update_ops += [delete_tag+'|'.join(u) for u in updates[:n_invalid_ops]] # invalid deletions

# write the operations to a file
with open(folder + "/ops_init.txt" , 'w') as op_file:
    for op in init_ops:
        op_file.write(op + '\n')
    
with open(folder + "/ops_update.txt" , 'w') as op_file:
    for op in update_ops:
        op_file.write(op + '\n')

with open(folder + "/ops_all.txt" , 'w') as op_file:
    for op in init_ops + update_ops:
        op_file.write(op + '\n')

print("Done!")

Processing nation.tbl
Processing customer.tbl
Processing orders.tbl
Processing lineitem.tbl
Done!


2. Split Dataset

In [2]:
folder = "src/main/resources/data"
file_to_split = "ops_init.txt"
split_ratio = [0.2, 0.4, 0.6, 0.8, 1.0]

def split_data(data, ratio):
    # randomly select a subset of the data
    n = int(len(data) * ratio)
    return random.sample(data, n)

with open(folder + f"/{file_to_split}", 'r') as op_file:
    data = op_file.readlines()
    subsets = [split_data(data, r) for r in split_ratio]
    for i, subset in enumerate(subsets):
        with open(folder + f"/{file_to_split.split('.txt')[0]}_{(i+1)*20}.txt", 'w') as op_file:
            for op in subset:
                op_file.write(op)


# Convert the operation to SQL insert/delete statements

In [3]:
fieldnames = {
    "nation": ["N_NATIONKEY", "N_NAME", "N_REGIONKEY", "N_COMMENT"],
    "customer": ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"],
    "orders": ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"],
    "lineitem": ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]
}

# 
def convert_to_sql(op_file):
    with open(op_file, 'r') as file:
        ops = file.readlines()
    insert_tag = "+"
    delete_tag = "-"
    sql_statements = []
    for op in ops:
        if op.startswith(insert_tag):
            table_name = op.split('|')[1]
            values = op.split('|')[2:]
            if "\n" in values:
                values.remove('\n')
            values = [f"'{v}'" if not v.isdigit() else v for v in values]
            values = [f"'{v}'" if v == "null" else v for v in values]
            sql_statement = f"INSERT INTO {table_name} VALUES ({', '.join(values)});"
            sql_statements.append(sql_statement)
        elif op.startswith(delete_tag):
            table_name = op.split('|')[1]
            condition = op.split('|')[2:]
            condition.remove('\n')
            condition = [f"{fieldnames[table_name][i]} = '{c}'" if not c.isdigit() else f"{fieldnames[table_name][i]} = {c}" for i, c in enumerate(condition)]
            sql_statement = f"DELETE FROM {table_name} WHERE {', '.join(condition)};"
            sql_statements.append(sql_statement)
    
    return sql_statements

# convert the ops_all.txt file to sql statements
folder = "src/main/resources/data"
fname = "ops_all.txt"
sql_statements = convert_to_sql(folder + "/" + fname, 'r')

# save the sql statements to a file
with open(folder + "/" + fname.replace("txt", "sql") , 'w') as sql_file:
    for statement in sql_statements:
        sql_file.write(statement + '\n')